# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""92748107410054…","""94590171303765…","""../../../../da…",1.2927e19,"""64921396585480…","""13459012231088…",false
"""12240071396895…","""15940073280510…","""../../../../da…",1.4856e19,"""64921396585480…","""23881597874743…",true
"""54044488605809…","""11356969929084…","""../../../../da…",5.6458e17,"""14108307185083…","""13459012231088…",false
"""12240071396895…","""61405775338278…","""../../../../da…",4.0723e18,"""37724848704482…","""13459012231088…",false
"""92748107410054…","""10871801798780…","""../../../../da…",1.4677e19,"""37724848704482…","""13908182357323…",true
"""54044488605809…","""17175497221760…","""../../../../da…",1.6330e19,"""58309060134148…",null,false
"""54044488605809…","""10201055502669…","""../../../../da…",8.7675e18,"""37724848704482…","""13908182357323…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""14854348474023…","""42439939960301…","""../../../../da…",1.2927e19,"""FirstMortgage3…","""12984705266642…","""Rejected"""
"""33200212055466…","""15200946957389…","""../../../../da…",1.4856e19,"""FirstMortgage3…","""14626649609009…","""Accepted"""
"""15226841170602…","""18270202527890…","""../../../../da…",5.6458e17,"""MoneyMarketSav…","""12984705266642…","""Rejected"""
"""33200212055466…","""14861922818461…","""../../../../da…",4.0723e18,"""BasicChecking""","""12984705266642…","""Rejected"""
"""14854348474023…","""23724012892267…","""../../../../da…",1.4677e19,"""BasicChecking""","""74860309640906…","""Accepted"""
"""15226841170602…","""15045386378629…","""../../../../da…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""15226841170602…","""50537264394300…","""../../../../da…",8.7675e18,"""BasicChecking""","""74860309640906…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""16010181789338…","""15980494866973…","""../../../../da…",1.2927e19,"""FirstMortgage3…","""98716385469007…","""Rejected"""
"""21203228628141…","""11173200428656…","""../../../../da…",1.4856e19,"""FirstMortgage3…","""10933737634131…","""Accepted"""
"""31829390812929…","""78070414630229…","""../../../../da…",5.6458e17,"""MoneyMarketSav…","""98716385469007…","""Rejected"""
"""21203228628141…","""13532799217763…","""../../../../da…",4.0723e18,"""BasicChecking""","""98716385469007…","""Rejected"""
"""16010181789338…","""10378725973445…","""../../../../da…",1.4677e19,"""BasicChecking""","""84363566523951…","""Accepted"""
"""31829390812929…","""13755189067750…","""../../../../da…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""31829390812929…","""18154363833382…","""../../../../da…",8.7675e18,"""BasicChecking""","""84363566523951…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""89240288844783…","""18124380068880…",1.2927e19,"""FirstMortgage3…","""15186244409906…","""Rejected"""
"""15779032907596…","""33418541079894…",1.4856e19,"""FirstMortgage3…","""68400999137430…","""Accepted"""
"""13054104807543…","""63115134587122…",5.6458e17,"""MoneyMarketSav…","""15186244409906…","""Rejected"""
"""15779032907596…","""90314686753266…",4.0723e18,"""BasicChecking""","""15186244409906…","""Rejected"""
"""89240288844783…","""97036858209217…",1.4677e19,"""BasicChecking""","""14884446698381…","""Accepted"""
"""13054104807543…","""28722924242918…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""13054104807543…","""26566094159259…",8.7675e18,"""BasicChecking""","""14884446698381…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'